# TKinter项目实战-屏保
### 项目分析
- 屏保可以自己启动，也可以手动启动
- 一旦敲击键盘或移动鼠标，或者其他的引发事件，则停止
- 如果屏保是一副画的话，则没有画框
- 图像的动作是随机的，具有随机性，可能包括数量，颜色，大小，运动方向，变形等
- 整个世界的构成：
    - ScreenSaver：
        - 需要一个canvas，大小与屏幕一致，没有边框
    - Ball：
        - 数量，颜色，大小，运动方向，变形等随机
        - 球能动，可以被调用

In [14]:
import random
import tkinter

class RandomBall():
    '''
    定义运动球的类
    '''
    def __init__(self, canvas, scrnwidth, scrnheight):
        '''
        canvas:画布，所有的内容都应该在画布上呈现出来，此处通过变量传入
        scrnwidth/scrnheight：屏幕宽高
        '''
        self.canvas = canvas
        # 球出现的初始位置要随机（x，y），此位置表示的是球的圆心！！！！
        self.x = random.randint(80, int(scrnwidth)-80)
        self.y = random.randint(80, int(scrnheight)-80)
        # 定义球运动的速度
        # 模拟运动，不断地擦掉原来的画，然后在一个新的地方再从新绘制
        self.xspeed = random.randint(8, 20)
        self.yspeed = random.randint(8, 20)
        # 定义屏幕的大小和高度
        self.scrnwidth = scrnwidth
        self.scrnheight = scrnheight
        # 球的大小随机
        # 此处球的大小用半径表示
        self.radius = random.randint(30, 70)
        # 定义颜色
        # RGB表示法，三个数字，每个数字的值是0-255之间，表示红绿蓝三个颜色的大小
        # 在某些系统中，直接用英文单词表示也可以，比如red，green
        # 此处用lambda表达式
        c = lambda: random.randint(0, 255)
        self.color = '#%02x%02x%02x'%(c(), c(), c())
        
    # 创建一个球
    def new_ball(self):
        '''
        用构造函数定义的变量值，在canvas上画一个球
        '''
        # tkinter没有画圆形的函数
        # 只有一个画椭圆的函数
        # 在一个长方形内画椭圆，我们只需要定义长方形左上角坐标(zs_x,zs_y)和右下角坐标(yx_x,yx_y)就好
        # 求两个坐标的方法自己思考
        zs_x = self.x - self.radius
        zs_y = self.y - self.radius
        yx_x = self.x + self.radius
        yx_y = self.y + self.radius
        # 在有两个坐标的情况下，就可以画圆啦
        # fill表示填充颜色
        # outline表示外围边框颜色
        self.item = self.canvas.create_oval(zs_x, zs_y, yx_x, yx_y, fill=self.color, outline=self.color)
        
    def move_ball(self):
        # 移动球的时候，需要控制球的方向
        # 每次球都有一个新的坐标
        self.x += self.xspeed
        self.y += self.yspeed
        # 判断是否撞墙
        # 撞墙就回头
        # 注意撞墙的算法判断
        if self.x + self.radius >= self.scrnwidth:
            self.xspeed = -self.xspeed
        elif self.x - self.radius <= 0:
            self.xspeed = -self.xspeed
        elif self.y + self.radius >= self.scrnheight:
            self.yspeed = -self.yspeed
        elif self.y - self.radius <= 0:
            self.yspeed = -self.yspeed
        # 在画布上移动
        self.canvas.move(self.item, self.xspeed, self.yspeed) 
        
        
class ScreenSaver():
    '''
    定义屏保的类
    可以被启动
    ''' 
    
    def __init__(self,):
        # 如何装随机产生的球？
        self.balls = []
        # 每次启动球的数量随机
        self.num_balls = random.randint(12, 18)
        # 生成root主窗口
        self.root = tkinter.Tk()
        # 取消边框
        self.root.overrideredirect(1)
        # 调整透明度
        self.root.attributes('-alpha', 1)
        # 任何鼠标移动都需要退出屏保
        self.root.bind("<Motion>", self.llrquit)
        # 同理，任何键盘移动也应该退出
        self.root.bind('<Any-Button>', self.llrquit)
        self.root.bind('<Key>', self.llrquit)
        # 得到屏幕的规格
        w,h = self.root.winfo_screenwidth(), self.root.winfo_screenheight()
        # 创建画布，包括画布的归属，规格
        self.canvas = tkinter.Canvas(self.root, width=w, height=h, bg="black")
        self.canvas.pack()
        
        # 在画布上画球
        for i in range(self.num_balls):
            ball = RandomBall(self.canvas, scrnwidth=w, scrnheight=h)
            ball.new_ball()
            self.balls.append(ball)
            
        self.run_screen_saver()
        self.root.mainloop()
        
    def run_screen_saver(self):
        for ball in self.balls:
            ball.move_ball()
        # after是200毫秒后启动一个函数，需要启动的函数是第二个参数
        self.canvas.after(50, self.run_screen_saver)
        
    def llrquit(self, e):
        # 此处只是利用了事件处理机制
        # 实际上并不关心事件的类型
        self.root.destroy()
if __name__=="__main__":
    # 自动屏保
    ScreenSaver()
